## 업종분류, 설립연도, 주소 추가

In [1]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys

In [2]:
# 파일 불러오기
data = pd.read_csv('./.gitignore/wanted_data.csv')

wan_corp_name = []

for q in data.기업명.apply(lambda x: re.sub(r'[(a-zA-Z)]', '', x)):
    wan_corp_name.append(re.sub('\s', '', q))

len(wan_corp_name)

1962

## 다트 크롤링

In [3]:
pro_extra_res = pd.DataFrame(columns=['기업명','주소','업종분류','설립연도','기업홈페이지URL'])

In [4]:
url = 'https://dart.fss.or.kr/dsae001/main.do#none'

driver= webdriver.Chrome()
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

for i in wan_corp_name:
    try:
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').send_keys(i)  # 키워드 작성
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        driver.find_element(By.XPATH, '//*[@id="rightWrap"]/div[1]/div[1]/ul/li[3]/a').click()
        time.sleep(.1)
        driver.find_element(By.XPATH, '//*[@id="corpTabel"]/tbody/tr/td[1]/span/a').click()

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기업명
        corp_name = i
        # 주소
        address = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[9]/td').text
        # 업종명
        sector = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[14]/td').text
        # 설립일
        corp_year = driver.find_element(By.XPATH, '//*[@id="corpDetailTabel"]/tbody/tr[15]/td').text
        # 기업 홈페이지
        corp_url = driver.find_element(By.XPATH, '//*[@id="homePage"]').text

        pro_extra_temp = pd.DataFrame([corp_name,address,sector,corp_year,corp_url], index=['기업명','주소','업종분류','설립연도','기업홈페이지URL']).T
        pro_extra_res = pd.concat([pro_extra_res, pro_extra_temp])

        # 키워드 지우기
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()
        time.sleep(.1)
        
    except:
        driver.find_element(By.XPATH, '//*[@id="textCrpNm"]').clear()

driver.quit()

In [6]:
pro_extra_res.to_csv('./wanted_dart_data.csv')